# CRIME IN LOS ANGELES
    Crime data from 2010 through September 2017

In [67]:
#TEAM MEMBERS:
#              Sylvester Zowonu
#              Frank Navarette
#              Max Ru
#Course      : CS 4661 (Data Science)
#Instructor  : Dr. Mohammad (Mo) Pourhomayoun 
#
#  
# 
# File Locations : Crime Data https://www.dropbox.com/s/rhnsu6p95clsojk/Crime_Data_2010_2017.csv?dl=0
#                  MO_Codes   https://www.dropbox.com/s/e3wuydmcgwto5ox/MO_Codes.csv?dl=0
# Instructions: Files should be in the same directory of this file

Import Packages

In [2]:
import pandas as pd
import numpy as np
import seaborn as sb
import datetime as dt
from pylab import rcParams
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [3]:
filename = 'Crime_Data_2010_2017.csv'
crime_data = pd.DataFrame()
crime_data = pd.read_csv(filename)

In [4]:
crime_data.head()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
0,1208575,03/14/2013,03/11/2013,1800,12,77th Street,1241,626,INTIMATE PARTNER - SIMPLE ASSAULT,0416 0446 1243 2000,...,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AO,Adult Other,626.0,NaN,NaN,NaN,6300 BRYNHURST AV,NaN,"(33.9829, -118.3338)"
1,102005556,01/25/2010,01/22/2010,2300,20,Olympic,2071,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,VAN NESS,15TH,"(34.0454, -118.3157)"
2,418,03/19/2013,03/18/2013,2030,18,Southeast,1823,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,200 E 104TH ST,NaN,"(33.942, -118.2717)"
3,101822289,11/11/2010,11/10/2010,1800,18,Southeast,1803,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,88TH,WALL,"(33.9572, -118.2717)"
4,42104479,01/11/2014,01/04/2014,2300,21,Topanga,2133,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329,...,NaN,IC,Invest Cont,745.0,NaN,NaN,NaN,7200 CIRRUS WY,NaN,"(34.2009, -118.6369)"


In [5]:
crime_data.shape

(1584316, 26)

# Crime types

In [6]:
crime_data['Crime Code Description'].value_counts().head(20)

BATTERY - SIMPLE ASSAULT                                        145767
VEHICLE - STOLEN                                                121329
BURGLARY FROM VEHICLE                                           121318
BURGLARY                                                        114751
THEFT PLAIN - PETTY ($950 & UNDER)                              113709
THEFT OF IDENTITY                                               100653
INTIMATE PARTNER - SIMPLE ASSAULT                                85908
VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS) 0114     79433
VANDALISM - MISDEAMEANOR ($399 OR UNDER)                         71523
ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT                   67631
THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)                  63995
ROBBERY                                                          63408
THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD0036     56377
CRIMINAL THREATS - NO WEAPON DISPLAYED                           44560
SHOPLI

# Count of Victim Gender

In [7]:
crime_data['Victim Sex'].value_counts()

M    739581
F    675402
X     24080
H        53
-         1
Name: Victim Sex, dtype: int64

# Count by Vicitim Race

In [8]:
Victims_bg = {
    "A": "Asian",
    "B": "Black",
    "C": "Chinese",
    "D": "Cambodian",
    "F": "Filipino",
    "G": "Guamanian",
    "H": "Hispanic/Latin/Mexican",
    "I": "American Indian/Alaskan Native",
    "J": "Japanese",
    "K": "Korean",
    "L": "Laotian",
    "O": "Other",
    "P": "Pacific Islander",
    "S": "Samoan",
    "U": "Hawaiian",
    "V": "Vietnamese",
    "W": "White",
    "X": "Unknown",
    "Z": "Asian Indian"
}
#crime_data["Victim Descent"] = crime_data["Victim Descent"].map(Victims_bg)
crime_data['Victim Descent'].value_counts()

H    549515
W    391855
B    255056
O    152776
X     41535
A     37143
K      7136
F      1885
I       663
C       618
P       276
J       237
U       137
V        86
G        61
Z        55
S        24
D        15
L        10
-         1
Name: Victim Descent, dtype: int64

# Type of Weapons used

In [9]:
crime_data['Weapon Description'].value_counts().head(20)

STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)    319818
VERBAL THREAT                                      43814
UNKNOWN WEAPON/OTHER WEAPON                        40746
HAND GUN                                           25352
SEMI-AUTOMATIC PISTOL                              10096
KNIFE WITH BLADE 6INCHES OR LESS                    9335
OTHER KNIFE                                         6946
UNKNOWN FIREARM                                     6037
VEHICLE                                             5348
REVOLVER                                            4567
BOTTLE                                              3806
BLUNT INSTRUMENT                                    3347
ROCK/THROWN OBJECT                                  3233
STICK                                               3200
CLUB/BAT                                            3155
FOLDING KNIFE                                       3065
SIMULATED GUN                                       2842
KITCHEN KNIFE                  

# Crime by periods [Day, Month, Year]

In [10]:
#convert date reported to date object
try:
    date_occ = [dt.datetime.strptime(d, "%m/%d/%Y").date() for d in crime_data["Date Occurred"]]
except:
    print("Error Date Format")
crime_data["Date Occurred"] = np.array(date_occ)

day = [data.isoweekday() for data in crime_data["Date Occurred"]]
mon = [data.month for data in crime_data["Date Occurred"]]
year = [data.year for data in crime_data["Date Occurred"]]

crime_data["Day Occurred"] = np.array(day)
crime_data["Month Occurred"] = np.array(mon)
crime_data["Year Occurred"] = np.array(year)

In [11]:
#Day of The Week
week_day = { 
    1:"Monday",
    2:"Tuesday",
    3:"Wednesday",
    4:"Thursday",
    5:"Friday",
    6:"Saturday",
    7:"Sunday"
}
#crime_data['Day Occurred'] = crime_data['Day Occurred'].map(week_day)
crime_data['Day Occurred'].value_counts()

5    244395
6    227804
1    225883
3    224344
4    222912
2    221931
7    217047
Name: Day Occurred, dtype: int64

In [12]:
#Month of The Year
month_year = { 
    1:"January",
    2:"February",
    3:"March",
    4:"April",
    5:"May",
    6:"June",
    7:"July",
    8:"August",
    9:"September",
    10:"October",
    11:"November",
    12:"December"
}
#crime_data['Month Occurred'] = crime_data['Month Occurred'].map(month_year)
crime_data['Month Occurred'].value_counts()

7     143091
8     142648
1     142503
5     141057
6     138645
3     138292
4     135144
10    123646
2     123471
9     122307
12    119256
11    114256
Name: Month Occurred, dtype: int64

In [13]:
crime_data['Month Occurred'].describe()[1:]

mean     6.343421
std      3.391193
min      1.000000
25%      3.000000
50%      6.000000
75%      9.000000
max     12.000000
Name: Month Occurred, dtype: float64

In [14]:
#Crime by the Years
crime_data['Year Occurred'].value_counts()

2016    223038
2015    213910
2010    208576
2012    200797
2011    200176
2014    194469
2013    191764
2017    151586
Name: Year Occurred, dtype: int64

In [15]:
crime_data['Victim Age'].describe()[1:]

mean    35.934195
std     16.811559
min     10.000000
25%     23.000000
50%     34.000000
75%     48.000000
max     99.000000
Name: Victim Age, dtype: float64

In [16]:
crime_data['Premise Description'].value_counts().head(20)

STREET                                          352160
SINGLE FAMILY DWELLING                          328198
MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)    204980
PARKING LOT                                     112576
SIDEWALK                                         79247
OTHER BUSINESS                                   71097
VEHICLE, PASSENGER/TRUCK                         60450
DRIVEWAY                                         32473
GARAGE/CARPORT                                   28407
DEPARTMENT STORE                                 22476
RESTAURANT/FAST FOOD                             20990
MARKET                                           16492
OTHER STORE                                      12653
YARD (RESIDENTIAL/BUSINESS)                      11113
PARKING UNDERGROUND/BUILDING                     10927
PARK/PLAYGROUND                                  10856
OTHER PREMISE                                    10674
HIGH SCHOOL                                      10389
ALLEY     

In [17]:
crime_data['Crime Code'].value_counts().head(20)

624    145767
510    121329
330    121318
310    114751
440    113709
354    100653
626     85908
740     79433
745     71523
230     67631
420     63995
210     63408
341     56377
930     44560
442     35032
331     22588
649     18856
946     16449
956     16371
900     16053
Name: Crime Code, dtype: int64

In [18]:
crime_data['Status Description'].value_counts().head(20)

Invest Cont     1227180
Adult Other      178175
Adult Arrest     162424
Juv Arrest        12619
Juv Other          3889
UNK                  29
Name: Status Description, dtype: int64

In [19]:
crime_data['Status Code'].value_counts().head(20)

IC    1227180
AO     178175
AA     162424
JA      12619
JO       3889
CC         24
TH          1
13          1
19          1
Name: Status Code, dtype: int64

OneHot Encoding function

In [20]:
def one_hot(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode 
    @return a DataFrame with one-hot encoding
    """
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    return df

In [21]:
crime_data['Area Name'].value_counts().head(20)
#list(crime_data)

77th Street    110605
Southwest      102259
N Hollywood     86405
Pacific         83763
Southeast       83517
Mission         80249
Northeast       76627
Van Nuys        75456
Newton          74385
Devonshire      74013
Topanga         73291
Hollywood       72239
Harbor          70954
Olympic         70133
West Valley     67797
Central         67096
Rampart         67010
West LA         66388
Wilshire        63598
Foothill        60939
Name: Area Name, dtype: int64

In [22]:
#crime_data['Area ID'].value_counts().head(20)

In [23]:
#crime_data['Reporting District'].value_counts().head(20)

In [24]:
crime_data.dtypes

DR Number                   int64
Date Reported              object
Date Occurred              object
Time Occurred               int64
Area ID                     int64
Area Name                  object
Reporting District          int64
Crime Code                  int64
Crime Code Description     object
MO Codes                   object
Victim Age                float64
Victim Sex                 object
Victim Descent             object
Premise Code              float64
Premise Description        object
Weapon Used Code          float64
Weapon Description         object
Status Code                object
Status Description         object
Crime Code 1              float64
Crime Code 2              float64
Crime Code 3              float64
Crime Code 4              float64
Address                    object
Cross Street               object
Location                   object
Day Occurred                int64
Month Occurred              int64
Year Occurred               int64
dtype: object

In [25]:
feature_cols = ['Area ID',
                'Day Occurred',
                'Month Occurred',
                'Year Occurred',
                'Time Occurred',
                'Reporting District',
                'Victim Age',
                'Victim Sex',
                'Victim Descent',
                'Premise Code',
                'Weapon Used Code',
                'Status Code']
X = crime_data[feature_cols]
X = one_hot(X,feature_cols)
X = X.drop(feature_cols,axis=1)
X.shape

(1584316, 3265)

In [26]:
y = crime_data['Crime Code']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

# KNN Model

In [28]:
knn = KNeighborsClassifier(n_neighbors=501) 

In [ ]:
knn.fit(X_train,y_train)

In [ ]:
predict_y = knn.predict(X_test)
score = accuracy_score(y_test, predict_y)

# Accuracy

In [ ]:
print('Accracy score {0:.2f}'.format(score*100))

# Random Forest Model

In [ ]:
rf_clf = RandomForestClassifier(n_jobs=2, random_state=0,verbose=1)
rf_clf.fit(X_train,y_train)

In [ ]:
y_predict_rf = rf_clf.predict(X_test)
score_rf = accuracy_score(y_test, y_predict_rf)

# Accuracy

In [ ]:
print('Accracy score {0:.2f}'.format(score_rf*100))

In [ ]:
rf_clf.feature_importances_